https://www.api.gov.uk/ons/open-geography-portal/#open-geography-portal

# Libraries

In [1]:
import numpy as np
import pandas as pd

import json
import urllib.request

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

## Shape files from ONS

https://geoportal.statistics.gov.uk/

https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services

## Retrieve layer data

In [3]:
url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Clinical_Commissioning_Groups_April_2019_Boundaries_EN_BUC_2022/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'

region_url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Regions_December_2022_EN_BUC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'

In [22]:
def create_geojson_url(dataset_name:str):
    url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/' + dataset_name + '/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'
    return url

In [5]:
def retrieve_geojson_from_url(url:str):
    with urllib.request.urlopen(url) as contents:
        geojson_data = json.loads(contents.read())
    
    shape_data = pd.json_normalize(geojson_data['features'])
    return shape_data

In [25]:
def save_geojson_from_url(url:str, filename:str):
    filename = filename + '.csv'
    shape_data = retrieve_geojson_from_url(url)
    shape_data.to_csv(filename, index=False)
    print(filename + ' successfully saved')

In [7]:
def save_geojson_from_dataset_name(dataset_name:str):
    url = create_geojson_url(dataset_name)
    save_geojson_from_url(url, dataset_name)

In [8]:
# with urllib.request.urlopen(url) as contents:
#     json_data = json.loads(contents.read())
# shape_data = pd.json_normalize(json_data['features'])

# shape_data.to_csv('test_shape_data.csv')

In [9]:
# with urllib.request.urlopen(region_url) as contents:
#     json_data = json.loads(contents.read())
# region_shape_data = pd.json_normalize(json_data['features'])

# region_shape_data.to_csv('test_region_shape_data.csv')

In [10]:
# with open('region/region_merged.geojson', 'r') as contents:
#     json_data = json.loads(contents.read())
# region_shape_data = pd.json_normalize(json_data['features'])
# region_shape_data.to_csv('region/region_merged_geojson_cleaned.csv')



## Shape directory

In [11]:
shape_directory_url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services?f=pjson'

with urllib.request.urlopen(shape_directory_url) as contents:
    shape_directory = json.loads(contents.read())
shape_directory = pd.json_normalize(shape_directory['services'])
shape_directory = shape_directory.loc[shape_directory['type']=='FeatureServer']

In [12]:
def shape_name_contains(str1:str, str2:str):
    str1 = str1.lower()
    str2 = str2.lower()
    return shape_directory[(shape_directory['name'].str.lower().str.contains(str1)) & (shape_directory['name'].str.lower().str.contains(str2))]

### icb

In [19]:
icb_shape_directory = shape_name_contains('icb_jul_2022', 'buc')
icb_shape_directory

,name,type,url
499,ICB_JUL_2022_EN_BUC_V3,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/ICB_JUL_2022_EN_BUC_V3/FeatureServer


In [26]:
# save_geojson_from_dataset_name(icb_shape_directory['name'].values[0])

ICB_JUL_2022_EN_BUC_V3.csv successfully saved


### sub_icb

In [27]:
sub_icb_shape_directory = pd.concat([shape_name_contains('sub_integrated_care_board', 'bgc'),
                                     shape_name_contains('sub_integrated_care_board', 'buc')
])

In [28]:
sub_icb_shape_directory

,name,type,url
1688,Sub_Integrated_Care_Board_Locations_April_2023_EN_BGC,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Sub_Integrated_Care_Board_Locations_April_2023_EN_BGC/FeatureServer


In [29]:
# save_geojson_from_dataset_name(sub_icb_shape_directory['name'].values[0])

Sub_Integrated_Care_Board_Locations_April_2023_EN_BGC.csv successfully saved


### ccg

In [30]:
ccg_shape_directory = pd.concat([shape_name_contains('buc', 'clinical_commissioning_groups'), # 2019, 2021
                                 shape_name_contains('generalised', 'clinical_commissioning_groups'), # 2020
                                 shape_name_contains('ugcb', 'ccg') # 2015 to 2018
])

In [31]:
ccg_shape_directory

,name,type,url
98,Clinical_Commissioning_Groups_April_2019_Boundaries_EN_BUC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Clinical_Commissioning_Groups_April_2019_Boundaries_EN_BUC_2022/FeatureServer
109,Clinical_Commissioning_Groups_April_2021_EN_BUC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Clinical_Commissioning_Groups_April_2021_EN_BUC_2022/FeatureServer
104,Clinical_Commissioning_Groups_April_2020_Ultra_Generalised_Boundaries_EN_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Clinical_Commissioning_Groups_April_2020_Ultra_Generalised_Boundaries_EN_2022/FeatureServer
52,CCG_Apr_2016_UGCB_in_England_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/CCG_Apr_2016_UGCB_in_England_2022/FeatureServer
55,CCG_Apr_2017_UGCB_in_England_V4_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/CCG_Apr_2017_UGCB_in_England_V4_2022/FeatureServer
57,CCG_Apr_2018_UGCB_in_England_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/CCG_Apr_2018_UGCB_in_England_2022/FeatureServer
61,CCG_July_2015_UGCB_in_England_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/CCG_July_2015_UGCB_in_England_2022/FeatureServer


In [34]:
# for i in ccg_shape_directory['name']:
#     # print(i)
#     save_geojson_from_dataset_name(i)

Clinical_Commissioning_Groups_April_2019_Boundaries_EN_BUC_2022.csv successfully saved
Clinical_Commissioning_Groups_April_2021_EN_BUC_2022.csv successfully saved
Clinical_Commissioning_Groups_April_2020_Ultra_Generalised_Boundaries_EN_2022.csv successfully saved
CCG_Apr_2016_UGCB_in_England_2022.csv successfully saved
CCG_Apr_2017_UGCB_in_England_V4_2022.csv successfully saved
CCG_Apr_2018_UGCB_in_England_2022.csv successfully saved
CCG_July_2015_UGCB_in_England_2022.csv successfully saved


### ltla

In [42]:
ltla_shape_directory = pd.concat([shape_name_contains('local_authority_districts', 'uk_buc') # 2016 to 2022
])

ltla_shape_directory

# shape_name_contains('local_authority_districts', 'generalised') # 2018
# shape_name_contains('local_authority_districts', 'ugcb') # 2019, 2020

,name,type,url
763,Local_Authority_Districts_April_2019_UK_BUC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Local_Authority_Districts_April_2019_UK_BUC_2022/FeatureServer
792,Local_Authority_Districts_December_2020_UK_BUC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Local_Authority_Districts_December_2020_UK_BUC_2022/FeatureServer
799,Local_Authority_Districts_December_2021_UK_BUC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Local_Authority_Districts_December_2021_UK_BUC_2022/FeatureServer
802,Local_Authority_Districts_December_2022_Boundaries_UK_BUC,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Local_Authority_Districts_December_2022_Boundaries_UK_BUC/FeatureServer
806,Local_Authority_Districts_May_2021_UK_BUC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Local_Authority_Districts_May_2021_UK_BUC_2022/FeatureServer
3134,Local_Authority_Districts_May_2023_UK_BUC,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Local_Authority_Districts_May_2023_UK_BUC/FeatureServer


In [43]:
# for i in ltla_shape_directory['name']:
#     # print(i)
#     save_geojson_from_dataset_name(i)

Local_Authority_Districts_April_2019_UK_BUC_2022.csv successfully saved
Local_Authority_Districts_December_2020_UK_BUC_2022.csv successfully saved
Local_Authority_Districts_December_2021_UK_BUC_2022.csv successfully saved
Local_Authority_Districts_December_2022_Boundaries_UK_BUC.csv successfully saved
Local_Authority_Districts_May_2021_UK_BUC_2022.csv successfully saved
Local_Authority_Districts_May_2023_UK_BUC.csv successfully saved


### utla

In [49]:
utla_shape_directory = pd.concat([shape_name_contains('counties_and_unitary_authorities', 'ew_buc'), # 2017, 2019
                                  shape_name_contains("counties_and_unitary_authorities_december_2018", 'gcb_ew'), # 2018
                                  shape_name_contains('counties_and_unitary_authorities', '2020_uk_bgc'), # 2020
                                  shape_name_contains('counties_and_unitary_authorities', 'en_buc'), # 2021
])

In [50]:
utla_shape_directory

,name,type,url
190,Counties_and_Unitary_Authorities_April_2019_Boundaries_EW_BUC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Counties_and_Unitary_Authorities_April_2019_Boundaries_EW_BUC_2022/FeatureServer
207,Counties_and_Unitary_Authorities_December_2017_EW_BUC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Counties_and_Unitary_Authorities_December_2017_EW_BUC_2022/FeatureServer
213,Counties_and_Unitary_Authorities_December_2018_GCB_EW_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Counties_and_Unitary_Authorities_December_2018_GCB_EW_2022/FeatureServer
220,Counties_and_Unitary_Authorities_December_2020_UK_BGC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Counties_and_Unitary_Authorities_December_2020_UK_BGC_2022/FeatureServer
222,Counties_and_Unitary_Authorities_December_2021_EN_BUC,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Counties_and_Unitary_Authorities_December_2021_EN_BUC/FeatureServer
227,Counties_and_Unitary_Authorities_December_2022_EN_BUC,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Counties_and_Unitary_Authorities_December_2022_EN_BUC/FeatureServer


In [51]:
# for i in utla_shape_directory['name']:
#     # print(i)
#     save_geojson_from_dataset_name(i)

Counties_and_Unitary_Authorities_April_2019_Boundaries_EW_BUC_2022.csv successfully saved
Counties_and_Unitary_Authorities_December_2017_EW_BUC_2022.csv successfully saved
Counties_and_Unitary_Authorities_December_2018_GCB_EW_2022.csv successfully saved
Counties_and_Unitary_Authorities_December_2020_UK_BGC_2022.csv successfully saved
Counties_and_Unitary_Authorities_December_2021_EN_BUC.csv successfully saved
Counties_and_Unitary_Authorities_December_2022_EN_BUC.csv successfully saved


### region

In [53]:
region_shape_directory = pd.concat([shape_name_contains('Regions_December_2022_EN_BUC', '')
])

region_shape_directory

,name,type,url
1582,Regions_December_2022_EN_BUC,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Regions_December_2022_EN_BUC/FeatureServer


In [54]:
# save_geojson_from_dataset_name(region_shape_directory['name'].values[0])

Regions_December_2022_EN_BUC.csv successfully saved


### STPs (pre Apr 2020) and STPs (from Apr 2020)

In [62]:
stp_shape_directory = pd.concat([shape_name_contains('sustainability_and_transformation', 'buc'), # 2018, 2021
                                 shape_name_contains('stp', '_gcb') # 2017, 2019, 2020
])

In [63]:
stp_shape_directory

,name,type,url
1697,Sustainability_and_Transformation_Partnerships_April_2018_EN_BUC_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Sustainability_and_Transformation_Partnerships_April_2018_EN_BUC_2022/FeatureServer
1703,Sustainability_and_Transformation_Partnerships_April_2021_EN_BUC_V2_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Sustainability_and_Transformation_Partnerships_April_2021_EN_BUC_V2_2022/FeatureServer
1667,STP_Apr_2019_GCB_in_England_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/STP_Apr_2019_GCB_in_England_2022/FeatureServer
1671,STP_Apr_2020_GCB_in_England_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/STP_Apr_2020_GCB_in_England_2022/FeatureServer
1675,STP_Feb_2017_GCB_in_England_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/STP_Feb_2017_GCB_in_England_2022/FeatureServer


In [64]:
# for i in stp_shape_directory['name']:
#     save_geojson_from_dataset_name(i)

Sustainability_and_Transformation_Partnerships_April_2018_EN_BUC_2022.csv successfully saved
Sustainability_and_Transformation_Partnerships_April_2021_EN_BUC_V2_2022.csv successfully saved
STP_Apr_2019_GCB_in_England_2022.csv successfully saved
STP_Apr_2020_GCB_in_England_2022.csv successfully saved
STP_Feb_2017_GCB_in_England_2022.csv successfully saved
